## Class Hierachies

### Abstract Classes

- The concept of inheritance exists in Scala like it does in all Object Oriented languages

- We have the usual notion of classes inheriting from abstract classes

- In an abstract class, you can either define a method fully, or just a method signature, and leave the implementation to be defined downstream
  - This is similar to "abstract method" in Python, and is known as an **abstract member** in Scala

- To inherit from base classes, use the key word `extends`

- Similar to the Python case, if `B` inherits from some abstract class `A` , then object of type `B` can be used wherever object of type `A` is required

- If no superclass is specified via `extends`, each new class definition is subclassed from the standard Java `Object` class

- Use `override` key word to override a method from a super class

  ```scala
    abstract class IntSet:
      def incl(x: Int): IntSet
      def contains(x: Int): Boolean

      def overrideMe(): Int = {
        1
      }
  ```

- Let's try an example; we want to implement the `IntSet` abstract class as binary trees

  ```scala
    class Empty() extends IntSet:
      def contains(x: Int): Boolean = False
      
      def incl(x: Int): IntSet = {
        NonEmpty(x, Empty(), Empty())
      } 

      override def overwriteMe(): Int = {
        2
      }

    end Empty

    class NonEmpty(elem: Int, left: IntSet, right: IntSet) extends IntSet:
      def contains(x: Int): Boolean = {
        if x < elem then left.contains(x)
        else if x > elem then right.contains(x)
        else true
      }

      def incl(x: Int): IntSet = {
        if x < elem then NonEmpty(elem, left.incl(x), right)
        else if x > elem then NonEmpty(elem, left, right.incl(x))
        else this
      }
  ```

- Interestingly, Scala also has built in method to define singleton object; that is, objects that you don't want to instantiate multiple times (either because instantiation is expensive, or serves no purpose besides cluttering up your memory)

- We can define **singleton** by using the `object` keyword

  ```scala
    object Empty extends IntSet:
      def contains(x: Int): Boolean = false
      def incl(x: Int): IntSet = NonEmpty(x, Empty, Empty)
    end Empty
  ```

### Scala Namespace

- Something to note about Scala is that there are 2 global namespaces to keep track of
  - A namespace for **types**
  - A namespace for **values**

- The implication here is that you can separate the definition of a type from the definition of the object. That is;
  ```scala
    class IntSet ...
    object IntSet:
      def singleton(x: Int) = NonEmpty(x, Empty, Empty)
  ```

- An object defined separately from the `class` is called a **companion object**

### Running Scala Programs

- To create a standalone Scala application, we can simply create an `object` that is called from the REPL

  ```scala
    object Hello:
      def main(args: Array[String]): Unit = println("hello world")

  ```

  ```bash
    > scala Hello
  ```

- Scala also has a convenience annotation `@main` that lets you run functions from command line while supplying args directly

  ```scala
    @main def birthday(name: String, age: Int) = {
      println(s"Hpbd ${name} you are ${age}")
    }
  ```

  ```bash
    > scala birthday Peter 100
  ```

### Exercise

- Write a method union for forming the union of two sets. You should
implement the following abstract class.

- See `1-slides.scala`

### Dynamic Binding

- Scala's ability to run methods based on the runtime type of the object that calls the method is known as **Dynamic Binding**

## Class Organisation

### Imports

- We have looked into how individual classes/objects are constructed in some detail. Now, we should discuss how multiple classes/objects are organised

- Scala allows you to define the package name on top of every `*.scala` file, which acts like a sort of identifier 

- You can use the package name to specify functions to import, and/or reference functions/objects

  ```scala
    package myScalaPackage

    def test() = {
      1
    }

    def test2() = {
      2
    }
  ```

  ```scala
    package myOtherScalaPackage

    import myScalaPackage.test //import single function
    import myScalaPackage.{test, test2} //import multiple
    import myScalaPackage._ //import everything

    val valid = test()
    val alsoValid = myScalaPackage.test()

  ```

### Traits

- Scala also enables a difference between inheritance and composition

- Inheritance
  - Previously, we looked at the `extends` keyword that allows you to create subclasses
  - Classes can only inherit from 1 superclass, so you cannot extend multiple classes

- Composition
  - The same keyword `extends` is used to do composition
  - The difference is that when you do `extends` from another class, the assumption is that you are specifying inheritance
  - To do composition, the superclass needs to be defined as a `trait` instead of an `abstract class`

- All Scala objects are subtypes of the `scala.Any` type 

- All reference types inherit from `scala.AnyRef`

- All primitive types inherit from `scala.AnyVal`

- At the bottom-most of Scala's type hierachy, there is the `Nothing` type, which is typically used for exceptions `Exc`

### Exercise

- What is the type of `if true then 1 else false`?

- You may think that this is `int` because the predicate always evaluates to true

- However, for stability, Scala deliberately considers BOTH paths of the if statement

- In this case, the statement could return a `Boolean`, or an `Int`, so the superclass `AnyVal` is the return type

## Polymorphism

- When defining anything in Scala, it is possible that a class/trait/function can be composed of different types on runtime
  - For example, if we define a `List`, we can say `List[Int]`, or `List[String]` etc

- For generality, if we don't know what type we will pass into the object, we can simply mark it with a generic type parameter `[T]`

- Scala doesn't use the type parameter for anything; it is generally inferred at run-time regardless (known as **type erasure**)
  - Nonetheless, it is good practise for some languages (C++, C#)
  ```scala
    def singleton[T](elem: T) = Cons[T](elem, Nil[T])

    singleton[Int](1) //valid
    singleton(1) //also valid
  ```

- The ability to accept multiple types in the same function is an example of  **polymorphism**

### Example: Create Immutable Linked List (Cons-List)

- A ConsList is a just a singly linked list
- Generally, it is made up from 2 building blocks;
  - `Nil` --> The empty list
  - `Cons` --> A cell containing an element and the remainder of the list

  ```scala
    //trait IntList ... //Not scalable. you need a FloatList, StringList etc
    trait List[T]: // So we define a List with a type parameter instead
      def isEmpty: Boolean
      def head: T
      def tail: List[T]

    class Cons[T](val head: T, val tail: List[T]) extends List[T]:
      def isEmpty = false
    
    class Nil[T] extends List[T]:
      def isEmpty = true
      def head = throw new NoSuchElementException(”Nil.head”)
      def tail = throw new NoSuchElementException(”Nil.tail”)

  ```

### Exercise

- Write a function nth that takes a list and an integer n and selects the n’th
element of the list.

  ```scala
    def nth[T](xs: List[T], n: Int): Int = ???
  ```

- Elements are numbered from 0.

- If index is outside the range from 0 up the the length of the list minus one, a `IndexOutOfBoundsException` should be thrown.

- See `3-slides.scala`

## Objects in Scala

- We'll try to define some Scala primitives from scratch as an exercise
  - Boolean
  - Int

- See `4-slides.scala`

## Functions as Objects

- So clearly, from the section above, primitives can be implemented as objects. What about functions?

- Functions are naturally objects in Scala

- For example, the functions below are equivalent

  ```scala
    package scala

    type A => B

    trait Function1[A, B]:
      def apply(x: A): B = {
        ???
      }

    trait Function2[A, B, C]: ???

    ...
  ```

- Even anonymous functions can be expanded to actual classes
  ```scala
    (x: Int) => x * x

    // same as
    new Function1[Int, Int]:
      def apply(x: Int) = x * x

    // same as
    { class $anonfun() extends Function1[Int, Int]:
        def apply(x: Int) = x * x
      $anonfun
    }
  ```

- Note the use of the keyword `apply`; when you call some function `f(a,b)`, you are implicitly doing `f.apply(a,b)`

- That is, the two code chunks below are equivalent
  ```scala
    val f = (x: Int) => x * x
    f(7)

    val f = new Function1[Int, Int]:
      def apply(x: Int) = x * x
    f.apply(7)
  ```

### Exercise

- See `5-slides.scala`